In [ ]:
from face_alignment import FaceAlignment, LandmarksType

from loreal_poc.dataloaders.loaders import DataLoader300W
from loreal_poc.dataloaders.wrappers import CroppedDataLoader, CachedDataLoader, BlurDataLoader
from loreal_poc.marks.facial_parts import FacialParts
from loreal_poc.visualisation.draw import draw_marks
from loreal_poc.tests.performance import NMEMean
from loreal_poc.models.wrappers import FaceAlignmentWrapper, OpenCVWrapper
from loreal_poc.tests.base import Test, TestDiff
import numpy as np

### Model and dataloader

In [ ]:
# model = FaceAlignmentWrapper(model=FaceAlignment(LandmarksType.TWO_D, device='cpu', flip_input=False))
model = OpenCVWrapper()
dl = DataLoader300W(dir_path="300W/sample", batch_size=1)

In [ ]:
blurred_dl = BlurDataLoader(dl, kernel_size=(15, 15), sigma=(9, 9))
# cropped_dl = CroppedDataLoader(dl, part=FacialParts.RIGHT_HALF.value)

original_image, marks, _ = dl[4]
blurred_image, _, _ = blurred_dl[4]

original_predictions = model.predict_image(original_image)
blurred_predictions = model.predict_image(blurred_image)
draw_marks(blurred_image, [marks, original_predictions, blurred_predictions], ["green", "red", "magenta"])

## Model predictions on cropped image

In [ ]:
test_result = TestDiff(metric=NMEMean, threshold=1).run(model, blurred_dl, dataloader_ref=dl)
test_result